In [1]:
import pandas as pd
import numpy as np
import os
import shutil

In [2]:
df_interactions = pd.read_csv('../data/userid_trackid_timestamp.tsv', sep='\t')
print(df_interactions.shape)
df_interactions.head()

(252984396, 3)


,user_id,track_id,timestamp
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38


In [3]:
df_interactions['period'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('M')
df_interactions = df_interactions[df_interactions['period'] == '2014-05']
print(df_interactions.shape)
df_interactions.head()

(3068523, 4)


,user_id,track_id,timestamp,period
56455621,20390,sE7pArXzW3mpEymq,2014-05-05 08:44:31,2014-05
56455622,20390,sE7pArXzW3mpEymq,2014-05-05 08:11:36,2014-05
56455623,20390,SDdw3A8XqMjjEYGt,2014-05-05 07:42:45,2014-05
56455624,20390,NRiXKeJztSerCkMS,2014-05-05 07:29:37,2014-05
56455625,20390,cwXiTYqrj7AOGn7x,2014-05-05 07:14:17,2014-05


In [4]:
# binarization: we convert the interactions to binary implicit feedback with a threshold of 2 on the interaction counts (reducing false-positive interactions)
def filter_interactions(listening_history: pd.DataFrame, min_interactions: int = 2, verbose: bool = True):
    lhs_count = listening_history.value_counts(subset=['user_id', 'track_id'])
    lhs_count = lhs_count[lhs_count >= min_interactions]
    listening_history = listening_history.set_index(['user_id', 'track_id']).loc[lhs_count.index]
    listening_history = listening_history.reset_index()
    return listening_history

def delete_duplicates(listening_history: pd.DataFrame):
    initial_shape = listening_history.shape
    listening_history = listening_history.drop_duplicates(subset=['user_id', 'track_id'], keep='first')
    print(f"Deleted {initial_shape[0] - listening_history.shape[0]} duplicates")
    return listening_history

df_interactions = filter_interactions(df_interactions, min_interactions=2)
df_interactions = delete_duplicates(df_interactions)
print(df_interactions.shape)
df_interactions.head()

Deleted 1577335 duplicates
(473815, 4)


,user_id,track_id,timestamp,period
0,117534,hxGBTxhSf7o6b1fz,2014-05-31 14:37:22,2014-05
2289,117534,fSQrzRe2g3gsEMoq,2014-05-31 14:52:28,2014-05
4576,117534,I53KHO4GCKMeUKUN,2014-05-31 14:44:53,2014-05
6863,117534,nd4vejLsgjjhl55B,2014-05-31 14:33:22,2014-05
9130,117534,WR82nGL7vKxitC5Q,2014-05-31 14:48:08,2014-05


In [5]:
# sparsity of dataset
n_users = df_interactions['user_id'].nunique()
n_items = df_interactions['track_id'].nunique()
n_interactions = df_interactions.shape[0]
sparsity = 1 - n_interactions / (n_users * n_items)
sparsity

0.9994916070473286

In [6]:
n_items

32355

In [7]:
n_users

28805

In [8]:
# export
df_interactions.to_csv('../data/userid_trackid_timestamp_onion.tsv', sep='\t', index=False)